# Python语言与经济大数据分析

# 第6_2讲 数据处理:逐日天气数据处理

#郭峰（上海财经大学投资系副教授）
#Email：guo.feng@mail.sufe.edu.cn
#2019-01-10
#主要任务：天气网数据19980101-20171231年，日*站点数据（820个站点，2018年站点减少），汇总为日*城市数据，进而汇总为月*城市、年*城市数据

In [9]:
#步骤1：先读取一个数据
import pandas as pd
import os
import numpy as np
import datetime
starttime = datetime.datetime.now()

#教学演示
#path='D:/python/郭峰Python讲义/数据与结果/逐日天气数据/'

#实际工作
path="D:/python/逐日天气数据/"

f1= open(path+'SURF_CLI_CHN_MUL_DAY-TEM-12001-201701.txt',encoding='gbk')
temp_site_daily=pd.read_table(f1,delim_whitespace=True,header=None,error_bad_lines=False) #读取txt文件的数字部分，无表头

#补充表头
temp_site_daily.columns =['site_id','site_lat','site_long','site_alt','year','month','daily','temp_mean','temp_max'
                          ,'temp_min','temp_mean_k','temp_max_k','temp_min_k']
#变量含义（气温，其他以此类推）依次为：
#站点代码、站点纬度、站点经度、站点海拔高度、年、月、日、日平均气温、日最高气温、日最低气温、平均气温质量控制码、最高气温质量控制码、最低低质量温控制码
#print(temp_site_daily.head())
print(temp_site_daily.max())

#异常变量更正
temp_site_daily.temp_mean[temp_site_daily['temp_mean']==32766]="NaN"
temp_site_daily.temp_mean[temp_site_daily['temp_mean']==32744]="NaN"

#更正变量单位
temp_site_daily['site_lat']=temp_site_daily['site_lat']/100
temp_site_daily['site_long']=temp_site_daily['site_long']/100
temp_site_daily['site_alt']=temp_site_daily['site_alt']/10
temp_site_daily['temp_mean']=temp_site_daily['temp_mean']/10
temp_site_daily['date']=temp_site_daily['year'].astype('str')+'-'+temp_site_daily['month'].astype('str')+'-'+temp_site_daily['daily'].astype('str')
temp_site_daily['date']=pd.to_datetime(temp_site_daily['date']).dt.date
print(temp_site_daily.max())

#只选一部分有用的变量
temp_site_daily=temp_site_daily[['site_id','date','temp_mean','temp_mean_k']]
print(temp_site_daily.head())

#temp_site_daily=temp_site_daily[temp_site_daily['temp_mean_k']==9]
print(temp_site_daily.shape)
print("变量长度:"+str(len(temp_site_daily))) #32列
endtime = datetime.datetime.now()
print((endtime - starttime).seconds)

site_id         59981
site_lat         5258
site_long       13258
site_alt       148000
year             2017
month               1
daily              31
temp_mean         258
temp_max          302
temp_min          251
temp_mean_k         9
temp_max_k          9
temp_min_k          0
dtype: int64
site_id             59981
site_lat            52.58
site_long          132.58
site_alt            14800
year                 2017
month                   1
daily                  31
temp_mean            25.8
temp_max              302
temp_min              251
temp_mean_k             9
temp_max_k              9
temp_min_k              0
date           2017-01-31
dtype: object
   site_id        date  temp_mean  temp_mean_k
0    50136  2017-01-01      -27.9            0
1    50136  2017-01-02      -30.0            0
2    50136  2017-01-03      -30.1            0
3    50136  2017-01-04      -29.3            0
4    50136  2017-01-05      -24.9            0
(26009, 4)
变量长度:26009
0


In [ ]:
#步骤2：批量读取某个数据
import pandas as pd
import os
import numpy as np
import datetime       
starttime = datetime.datetime.now()

path_wind='D:/python/郭峰Python讲义/数据与结果/逐日天气数据/风向风速/'
files_wind= os.listdir(path_wind)  #得到文件夹下的所有文件名称
wind_site_daily=pd.DataFrame(data=None,columns=range(0,17)) #日照时数变量列数不一样
for file in files_wind:    
    f1= open(path_wind+file,encoding='gbk')
    f2=pd.read_table(f1,delim_whitespace=True,header=None,error_bad_lines=False) #读取txt文件的数字部分，无表头
    wind_site_daily= wind_site_daily.append(f2)
wind_site_daily.columns =['site_id','site_lat','site_long','site_alt','year','month','daily','wind_mean','wind_max','wind_max_direction','wind_extreme','wind_extreme_direction','wind_mean_k','wind_max_k','wind_max_direction_k','wind_extreme_k','wind_extreme_direction_k']
wind_site_daily['date']=wind_site_daily['year'].astype('str')+'-'+wind_site_daily['month'].astype('str')+'-'+wind_site_daily['daily'].astype('str')
wind_site_daily['date']=pd.to_datetime(wind_site_daily['date']).dt.date
wind_site_daily=wind_site_daily[['site_id','date','wind_mean','wind_max','wind_mean_k','wind_max_k','wind_max_direction','wind_max_direction_k']]
wind_site_daily.drop_duplicates(subset=['site_id','date'],keep='first',inplace=True) 
print(wind_site_daily.shape)

endtime = datetime.datetime.now()
print((endtime - starttime).seconds)

In [ ]:
#步骤3：批量读取所有天气数据
import pandas as pd
import os
import numpy as np
import datetime       
starttime = datetime.datetime.now()

#教学演示
path1='D:/python/郭峰Python讲义/数据与结果/逐日天气数据/'

#实际工作
#path1="D:/python/逐日天气数据/"

#气温
path_temp='D:/python/郭峰Python讲义/数据与结果/逐日天气数据/气温/'
#path_temp="D:/python/逐日天气数据/气温/"
files_temp= os.listdir(path_temp)  #得到文件夹下的所有文件名称
temp_site_daily=pd.DataFrame(data=None,columns=range(0,13)) #先生成一个空的数据，就不用先读取再删除了
for file in files_temp:    
    f1= open(path_temp+file,encoding='gbk')
    f2=pd.read_table(f1,delim_whitespace=True,header=None,error_bad_lines=False) #读取txt文件的数字部分，无表头
    temp_site_daily= temp_site_daily.append(f2)
temp_site_daily.columns =['site_id','site_lat','site_long','site_alt','year','month','daily','temp_mean','temp_max','temp_min','temp_mean_k','temp_max_k','temp_min_k']
#站点代码、站点纬度、站点经度、站点海拔高度、年、月、日、日平均气温、日最高气温、日最低气温、平均气温质量控制码、最高气温质量控制码、最低低质量温控制码
temp_site_daily['date']=temp_site_daily['year'].astype('str')+'-'+temp_site_daily['month'].astype('str')+'-'+temp_site_daily['daily'].astype('str')
temp_site_daily['date']=pd.to_datetime(temp_site_daily['date']).dt.date
temp_site_daily=temp_site_daily[['site_id','site_lat','site_long','site_alt','date','temp_mean','temp_mean_k']]
temp_site_daily.drop_duplicates(subset=['site_id','date'],keep='first',inplace=True) 
print(temp_site_daily.shape)

#地温
path_gtemp='D:/python/郭峰Python讲义/数据与结果/逐日天气数据/地温/'
#path_gtemp="D:/python/逐日天气数据/地温/"
files_gtemp= os.listdir(path_gtemp)  #得到文件夹下的所有文件名称
gtemp_site_daily=pd.DataFrame(data=None,columns=range(0,13)) #先生成一个空的数据，就不用先读取再删除了
for file in files_gtemp:    
    f1= open(path_gtemp+file,encoding='gbk')
    f2=pd.read_table(f1,delim_whitespace=True,header=None,error_bad_lines=False) #读取txt文件的数字部分，无表头
    gtemp_site_daily= gtemp_site_daily.append(f2)
gtemp_site_daily.columns =['site_id','site_lat','site_long','site_alt','year','month','daily','gtemp_mean','gtemp_max','gtemp_min','gtemp_mean_k','gtemp_max_k','gtemp_min_k']
gtemp_site_daily['date']=gtemp_site_daily['year'].astype('str')+'-'+gtemp_site_daily['month'].astype('str')+'-'+gtemp_site_daily['daily'].astype('str')
gtemp_site_daily['date']=pd.to_datetime(gtemp_site_daily['date']).dt.date
gtemp_site_daily=gtemp_site_daily[['site_id','date','gtemp_mean','gtemp_mean_k']]
gtemp_site_daily.drop_duplicates(subset=['site_id','date'],keep='first',inplace=True) 
print(gtemp_site_daily.shape)

#降水
path_rain="D:/python/郭峰Python讲义/数据与结果/逐日天气数据/降水/"
#path_rain="D:/python/逐日天气数据/降水/"
files_rain= os.listdir(path_rain)  #得到文件夹下的所有文件名称
rain_site_daily=pd.DataFrame(data=None,columns=range(0,13)) #先生成一个空的数据，就不用先读取再删除了
for file in files_rain:    
    f1= open(path_rain+file,encoding='gbk')
    f2=pd.read_table(f1,delim_whitespace=True,header=None,error_bad_lines=False) #读取txt文件的数字部分，无表头
    rain_site_daily= rain_site_daily.append(f2)
rain_site_daily.columns =['site_id','site_lat','site_long','site_alt','year','month','daily','rain1','rain2','rain3','rain1_k','rain2_k','rain3_k']
rain_site_daily['date']=rain_site_daily['year'].astype('str')+'-'+rain_site_daily['month'].astype('str')+'-'+rain_site_daily['daily'].astype('str')
rain_site_daily['date']=pd.to_datetime(rain_site_daily['date']).dt.date
rain_site_daily=rain_site_daily[['site_id','date','rain3','rain3_k']]
rain_site_daily.drop_duplicates(subset=['site_id','date'],keep='first',inplace=True) 
print(rain_site_daily.shape)

#日照时数
path_sunshine="D:/python/郭峰Python讲义/数据与结果/逐日天气数据/日照/"
#path_sunshine="D:/python/逐日天气数据/日照/"
files_sunshine= os.listdir(path_sunshine)  #得到文件夹下的所有文件名称
sunshine_site_daily=pd.DataFrame(data=None,columns=range(0,9)) #日照时数变量列数不一样
for file in files_sunshine:    
    f1= open(path_sunshine+file,encoding='gbk',errors="ignore")
    f2=pd.read_table(f1,delim_whitespace=True,header=None,error_bad_lines=False) #读取txt文件的数字部分，无表头
    sunshine_site_daily= sunshine_site_daily.append(f2)
sunshine_site_daily.columns =['site_id','site_lat','site_long','site_alt','year','month','daily','sunshine','sunshine_k']
sunshine_site_daily.dropna(subset=["daily"],inplace=True)  #删除作者为空的样本
sunshine_site_daily['daily']=sunshine_site_daily['daily'].astype(int)
sunshine_site_daily['date']=sunshine_site_daily['year'].astype('str')+'-'+sunshine_site_daily['month'].astype('str')+'-'+sunshine_site_daily['daily'].astype('str')
sunshine_site_daily['date']=pd.to_datetime(sunshine_site_daily['date']).dt.date
sunshine_site_daily=sunshine_site_daily[['site_id','date','sunshine','sunshine_k']]
sunshine_site_daily.drop_duplicates(subset=['site_id','date'],keep='first',inplace=True) 
print(sunshine_site_daily.shape)

#相对湿度
path_humidity="D:/python/郭峰Python讲义/数据与结果/逐日天气数据/相对湿度/"
#path_humidity="D:/python/逐日天气数据/相对湿度/"
files_humidity= os.listdir(path_humidity)  #得到文件夹下的所有文件名称
humidity_site_daily=pd.DataFrame(data=None,columns=range(0,11)) #日照时数变量列数不一样
for file in files_humidity:    
    f1= open(path_humidity+file,encoding='gbk')
    f2=pd.read_table(f1,delim_whitespace=True,header=None,error_bad_lines=False) #读取txt文件的数字部分，无表头
    humidity_site_daily= humidity_site_daily.append(f2)
humidity_site_daily.columns =['site_id','site_lat','site_long','site_alt','year','month','daily','humidity','humidity_min','humidity_k','humidity_min_k']
humidity_site_daily['date']=humidity_site_daily['year'].astype('str')+'-'+humidity_site_daily['month'].astype('str')+'-'+humidity_site_daily['daily'].astype('str')
humidity_site_daily['date']=pd.to_datetime(humidity_site_daily['date']).dt.date
humidity_site_daily=humidity_site_daily[['site_id','date','humidity','humidity_k']]
humidity_site_daily.drop_duplicates(subset=['site_id','date'],keep='first',inplace=True) 
print(humidity_site_daily.shape)

#气压
path_pressure="D:/python/郭峰Python讲义/数据与结果/逐日天气数据/气压/"
#path_pressure="D:/python/逐日天气数据/气压/"
files_pressure= os.listdir(path_pressure)  #得到文件夹下的所有文件名称
pressure_site_daily=pd.DataFrame(data=None,columns=range(0,13)) #日照时数变量列数不一样
for file in files_pressure:    
    f1= open(path_pressure+file,encoding='gbk')
    f2=pd.read_table(f1,delim_whitespace=True,header=None,error_bad_lines=False) #读取txt文件的数字部分，无表头
    pressure_site_daily= pressure_site_daily.append(f2)
pressure_site_daily.columns =['site_id','site_lat','site_long','site_alt','year','month','daily','pressure_mean','pressure_max','pressure_min','pressure_mean_k','pressure_max_k','pressure_min_k']
pressure_site_daily['date']=pressure_site_daily['year'].astype('str')+'-'+pressure_site_daily['month'].astype('str')+'-'+pressure_site_daily['daily'].astype('str')
pressure_site_daily=pressure_site_daily[pressure_site_daily['daily']<=31] #存在一个错误
pressure_site_daily['date']=pd.to_datetime(pressure_site_daily['date']).dt.date
pressure_site_daily=pressure_site_daily[['site_id','date','pressure_mean','pressure_mean_k']]
pressure_site_daily.drop_duplicates(subset=['site_id','date'],keep='first',inplace=True) 
print(pressure_site_daily.shape)

#蒸发
path_evaporation="D:/python/郭峰Python讲义/数据与结果/逐日天气数据/蒸发/"
#path_evaporation="D:/python/逐日天气数据/蒸发/"
files_evaporation= os.listdir(path_evaporation)  #得到文件夹下的所有文件名称
evaporation_site_daily=pd.DataFrame(data=None,columns=range(0,11)) #日照时数变量列数不一样
for file in files_evaporation:    
    f1= open(path_evaporation+file,encoding='gbk')
    f2=pd.read_table(f1,delim_whitespace=True,header=None,error_bad_lines=False) #读取txt文件的数字部分，无表头
    evaporation_site_daily= evaporation_site_daily.append(f2)
evaporation_site_daily.columns =['site_id','site_lat','site_long','site_alt','year','month','daily','evaporation_big','evaporation_small','evaporation_big_k','evaporation_small_k']
evaporation_site_daily['date']=evaporation_site_daily['year'].astype('str')+'-'+evaporation_site_daily['month'].astype('str')+'-'+evaporation_site_daily['daily'].astype('str')
evaporation_site_daily['date']=pd.to_datetime(evaporation_site_daily['date']).dt.date
evaporation_site_daily=evaporation_site_daily[['site_id','date','evaporation_big','evaporation_small','evaporation_big_k','evaporation_small_k']]
evaporation_site_daily.drop_duplicates(subset=['site_id','date'],keep='first',inplace=True) 
print(evaporation_site_daily.shape)

#风向风速
path_wind="D:/python/郭峰Python讲义/数据与结果/逐日天气数据/风向风速/"
#path_wind="D:/python/逐日天气数据/风向风速/"
files_wind= os.listdir(path_wind)  #得到文件夹下的所有文件名称
wind_site_daily=pd.DataFrame(data=None,columns=range(0,17)) #日照时数变量列数不一样
for file in files_wind:    
    f1= open(path_wind+file,encoding='gbk')
    f2=pd.read_table(f1,delim_whitespace=True,header=None,error_bad_lines=False) #读取txt文件的数字部分，无表头
    wind_site_daily= wind_site_daily.append(f2)
wind_site_daily.columns =['site_id','site_lat','site_long','site_alt','year','month','daily','wind_mean','wind_max','wind_max_direction','wind_extreme','wind_extreme_direction','wind_mean_k','wind_max_k','wind_max_direction_k','wind_extreme_k','wind_extreme_direction_k']
wind_site_daily['date']=wind_site_daily['year'].astype('str')+'-'+wind_site_daily['month'].astype('str')+'-'+wind_site_daily['daily'].astype('str')
wind_site_daily['date']=pd.to_datetime(wind_site_daily['date']).dt.date
wind_site_daily=wind_site_daily[['site_id','date','wind_mean','wind_max','wind_mean_k','wind_max_k','wind_max_direction','wind_max_direction_k']]
wind_site_daily.drop_duplicates(subset=['site_id','date'],keep='first',inplace=True) 
print(wind_site_daily.shape)

weather_site_daily=pd.merge(temp_site_daily,gtemp_site_daily,on=['site_id','date'],how='left')
weather_site_daily=pd.merge(weather_site_daily,rain_site_daily,on=['site_id','date'],how='left')
weather_site_daily=pd.merge(weather_site_daily,sunshine_site_daily,on=['site_id','date'],how='left')
weather_site_daily=pd.merge(weather_site_daily,humidity_site_daily,on=['site_id','date'],how='left')
weather_site_daily=pd.merge(weather_site_daily,pressure_site_daily,on=['site_id','date'],how='left')
weather_site_daily=pd.merge(weather_site_daily,evaporation_site_daily,on=['site_id','date'],how='left')
weather_site_daily=pd.merge(weather_site_daily,wind_site_daily,on=['site_id','date'],how='left')

weather_site_daily.to_csv(path1+'weather_site_daily.csv',encoding='utf8')
print(weather_site_daily.shape)
print(weather_site_daily.head())

endtime = datetime.datetime.now()
print((endtime - starttime).seconds)

In [ ]:
#步骤4：数据清理,异常数据处理
#清理的还不干净
import pandas as pd
import os
import numpy as np
import datetime       
starttime = datetime.datetime.now()
path1='D:/python/郭峰Python讲义/数据与结果/逐日天气数据/'
#path1="D:/python/逐日天气数据/"
f1 = open(path1+"weather_site_daily.csv",encoding='utf8')
weather_site_daily= pd.read_csv(f1,header=0,sep=',')
print(weather_site_daily[0:10])
print(weather_site_daily.max())

#异常变量标识的替换
weather_site_daily.temp_mean[weather_site_daily['temp_mean'].astype(float)==32766]="NaN"
weather_site_daily.temp_mean[weather_site_daily['temp_mean'].astype(float)==32744]="NaN"
weather_site_daily.gtemp_mean[weather_site_daily['gtemp_mean'].astype(float)==32766]="NaN"
weather_site_daily.gtemp_mean[weather_site_daily['gtemp_mean'].astype(float)==32744]="NaN"
weather_site_daily.rain3[weather_site_daily['rain3'].astype(float)==32766]="NaN"
weather_site_daily.rain3[weather_site_daily['rain3'].astype(float)==32744]="NaN"
weather_site_daily.rain3[weather_site_daily['rain3'].astype(float)==32700]=0
weather_site_daily.sunshine[weather_site_daily['sunshine'].astype(float)==32766]="NaN"
weather_site_daily.sunshine[weather_site_daily['sunshine'].astype(float)==32744]="NaN"
weather_site_daily.humidity[weather_site_daily['humidity'].astype(float)==32766]="NaN"
weather_site_daily.humidity[weather_site_daily['humidity'].astype(float)==32744]="NaN"
weather_site_daily.pressure_mean[weather_site_daily['pressure_mean'].astype(float)==32766]="NaN"
weather_site_daily.pressure_mean[weather_site_daily['pressure_mean'].astype(float)==32744]="NaN"
weather_site_daily.evaporation_big[weather_site_daily['evaporation_big'].astype(float)==32766]="NaN"
weather_site_daily.evaporation_big[weather_site_daily['evaporation_big'].astype(float)==32744]="NaN"
weather_site_daily.evaporation_small[weather_site_daily['evaporation_small'].astype(float)==32766]="NaN"
weather_site_daily.evaporation_small[weather_site_daily['evaporation_small'].astype(float)==32744]="NaN"
weather_site_daily.wind_mean[weather_site_daily['wind_mean'].astype(float)==32766]="NaN"
weather_site_daily.wind_mean[weather_site_daily['wind_mean'].astype(float)==32744]="NaN"
weather_site_daily.wind_max[weather_site_daily['wind_max'].astype(float)==32766]="NaN"
weather_site_daily.wind_max[weather_site_daily['wind_max'].astype(float)==32744]="NaN"
weather_site_daily.wind_max_direction[weather_site_daily['wind_max_direction'].astype(float)==32744]="NaN"
weather_site_daily.wind_max_direction[weather_site_daily['wind_max_direction'].astype(float)==32766]="NaN"
weather_site_daily.rain3[weather_site_daily['rain3'].astype(float)>=32744]="NaN"
weather_site_daily.sunshine[weather_site_daily['sunshine'].astype(float)>=32744]="NaN"
print(weather_site_daily.max())
print(weather_site_daily['temp_mean'].astype(float).max())

#变量单位的调整
weather_site_daily['temp_mean']=weather_site_daily['temp_mean'].astype(float)/10
weather_site_daily['gtemp_mean']=weather_site_daily['gtemp_mean'].astype(float)/10
weather_site_daily['sunshine']=weather_site_daily['sunshine'].astype(float)/10
weather_site_daily['pressure_mean']=weather_site_daily['pressure_mean'].astype(float)/10
weather_site_daily['evaporation_big']=weather_site_daily['evaporation_big'].astype(float)/10
weather_site_daily['evaporation_small']=weather_site_daily['evaporation_small'].astype(float)/10
weather_site_daily['wind_mean']=weather_site_daily['wind_mean'].astype(float)/10
weather_site_daily['wind_max']=weather_site_daily['wind_max'].astype(float)/10
weather_site_daily['rain3']=weather_site_daily['rain3'].astype(float)/10

 
print(weather_site_daily.max())
print(weather_site_daily.min())
print(weather_site_daily.shape)

weather_site_daily.to_csv(path1+'weather_site_daily_clean.csv',encoding='utf8')
endtime = datetime.datetime.now()
print((endtime - starttime).seconds)
#站点级数据到此就可以用了

In [1]:
#步骤5：城市和站点的匹配
#城市的天气一距离该城市最近的站点为准
#距离根据经纬度计算，站点经纬度给定，城市经纬度自己搜集，以城市政府所在地为准
#这个要运行2个多小时
from math import radians, cos, sin, asin, sqrt
import pandas as pd
import os
import numpy as np
import datetime       
starttime = datetime.datetime.now()
#path1='D:/python/郭峰Python讲义/数据与结果/逐日天气数据/'
path1="D:/python/逐日天气数据/"

#2018年站点缺失200多个站点（也有之前没有的站点50多个），从而2018年的站点需要单独匹配
#f2 = open(path1+"站点列表.csv",encoding='utf8')
#sites= pd.read_csv(f2,header=0,sep=',')

f2 = open(path1+"站点列表2018.csv",encoding='utf8')
sites= pd.read_csv(f2,header=0,sep=',')

f3 = open(path1+"地市经纬度.csv",encoding='utf8')
citys= pd.read_csv(f3,header=0,sep=',')

def haversine(lon1,lat1,lon2,lat2): # 经度1，纬度1，经度2，纬度2 （十进制度数）
    # 将十进制度数转化为弧度
    lon1,lat1,lon2,lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # haversine公式
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # 地球平均半径，单位为公里
    return c * r
    
citys['dist']=0
citys['site_id']=0
citys['site_name']=0
sites['dist']=0    
for i in range(0,len(citys)):
    for j in range(0,len(sites)):
        sites['dist'][j]=haversine(sites['site_long'][j],sites['site_lat'][j],citys['city_long'][i],citys['city_lat'][i])
    citys['dist'][i]=sites['dist'].min(axis=0)  #所有站点中的最小值
    citys['site_id'][i]=sites['site_id'][sites['dist'].argmin(axis=0)] #sites['dist'].argmin(axis=0) 为所有站点中距离最小值对应的索引
    #citys['site_name'][i]=sites['site_name'][sites['dist'].argmin(axis=0)] 
    print(i) #查看程序循环到哪里了

print(sites[0:10])
print(citys[0:10])

citys.to_csv(path1+'city_site_id2.csv',encoding='utf8')
 
endtime = datetime.datetime.now()
print((endtime - starttime).seconds)

D:\python\WinPython-64bit-3.6.2.0Qt5\python-3.6.2.amd64\lib\site-packages\ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\python\WinPython-64bit-3.6.2.0Qt5\python-3.6.2.amd64\lib\site-packages\ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\python\WinPython-64bit-3.6.2.0Qt5\python-3.6.2.amd64\lib\site-packages\ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [4]:
#步骤6：城市-日天气数据构造
import pandas as pd
import os
import numpy as np
import datetime       
starttime = datetime.datetime.now()
#path1='D:/python/郭峰Python讲义/数据与结果/逐日天气数据/'
path1="D:/python/逐日天气数据/"

f1 = open(path1+"weather_site_daily_clean.csv",encoding='utf8')
weather_site_daily= pd.read_csv(f1,header=0,sep=',')
print(weather_site_daily.head())
weather_site_daily['date'] = pd.to_datetime(weather_site_daily['date'])
weather_site_daily['year']=weather_site_daily['date'].dt.year
weather_site_daily13_17=weather_site_daily[weather_site_daily['year']<2018]
weather_site_daily18=weather_site_daily[weather_site_daily['year']>2017]

f2 = open(path1+"city_site_id.csv",encoding='utf8')
city_site_id= pd.read_csv(f2,header=0,sep=',')
f3 = open(path1+"city_site_id2.csv",encoding='utf8')
city_site_id2= pd.read_csv(f3,header=0,sep=',')

weather_city_daily13_17=pd.merge(weather_site_daily13_17,city_site_id,on=['site_id'],how='left')
weather_city_daily18=pd.merge(weather_site_daily18,city_site_id2,on=['site_id'],how='left')

weather_city_daily= weather_city_daily13_17.append(weather_city_daily18)

weather_city_daily.dropna(subset=["city_code"],inplace=True)  #删除没有分配给城市的站点
weather_city_daily=weather_city_daily[weather_city_daily['sunshine']<20] #有两个样本存在大于20小时的日照时间
weather_city_daily=weather_city_daily[['city_name','city_code','date','site_id','temp_mean','gtemp_mean','rain3',
                                      'sunshine','humidity','wind_mean','wind_max','wind_max_direction']]
weather_city_daily.rename(columns={'temp_mean':'temp', 'gtemp_mean':'gtemp','rain3':'rain','wind_mean':'wind','wind_max_direction':'wind_d'}, inplace = True)
weather_city_daily.to_csv(path1+'weather_city_daily.csv',encoding='utf8',index=False)
print(weather_city_daily.max())
print(weather_city_daily.min())
print(weather_city_daily.shape)

endtime = datetime.datetime.now()
print((endtime - starttime).seconds)


   Unnamed: 0  Unnamed: 0.1  site_id  site_lat  site_long  site_alt  \
0           0             0    50136      5258      12231      4330   
1           1             1    50136      5258      12231      4330   
2           2             2    50136      5258      12231      4330   
3           3             3    50136      5258      12231      4330   
4           4             4    50136      5258      12231      4330   

         date  temp_mean  temp_mean_k  gtemp_mean          ...           \
0  1998-01-01      -41.7          0.0       -43.2          ...            
1  1998-01-02      -39.2          0.0       -40.7          ...            
2  1998-01-03      -36.4          0.0       -38.4          ...            
3  1998-01-04      -40.2          0.0       -41.2          ...            
4  1998-01-05      -38.3          0.0       -39.3          ...            

   evaporation_big  evaporation_small  evaporation_big_k  evaporation_small_k  \
0              0.0                NaN    

In [5]:
#步骤6：城市-月和城市-年天气数据构造
import pandas as pd
import os
import numpy as np
import datetime       
starttime = datetime.datetime.now()

path1='D:/python/郭峰Python讲义/数据与结果/逐日天气数据/'

#path1="D:/python/逐日天气数据/"

f1 = open(path1+"weather_city_daily.csv",encoding='utf8')
weather_city_daily= pd.read_csv(f1,header=0,sep=',')

weather_city_daily['date'] = pd.to_datetime(weather_city_daily['date'])
weather_city_daily['year']=weather_city_daily['date'].dt.year
weather_city_daily['month']=weather_city_daily['date'].dt.month

group=weather_city_daily.groupby(['city_name',"city_code",'year','month'])
weather_city_month=pd.DataFrame(columns=['temp','gtemp'])
weather_city_month['temp']=group['temp'].mean()
weather_city_month['gtemp']=group['gtemp'].mean()
weather_city_month['rain']=group['rain'].mean()
weather_city_month['wind']=group['wind'].mean()
weather_city_month['sunshine']=group['sunshine'].mean()
weather_city_month['humidity']=group['humidity'].mean()

weather_city_month.to_csv(path1+'weather_city_month.csv',encoding='utf8')
ftemp = open(path1+"weather_city_month.csv",encoding='utf8')
weather_city_month= pd.read_csv(ftemp,header=0,sep=',')
weather_city_month.to_csv(path1+'weather_city_month.csv',encoding='utf8')
print(weather_city_month.shape)
print(weather_city_month.head())

#年度天气，可以考虑用年度均值，也可以用1月份均值或7月份均值,这里用的是1月均值

weather_city_year=weather_city_month[weather_city_month['month']<2]
print(weather_city_year.shape)
print(weather_city_year.head())
print(weather_city_year.max())
print(weather_city_year.min())
weather_city_year=weather_city_year[['city_name','city_code','year','temp','gtemp','rain','wind',
                                   'sunshine','humidity']]
weather_city_year.to_csv(path1+'weather_city_year.csv',encoding='utf8',index=False)

endtime = datetime.datetime.now()
print((endtime - starttime).seconds)

(671, 10)
  city_name  city_code  year  month       temp      gtemp      rain      wind  \
0      七台河市     2309.0  2017      1 -13.916667  -9.020000  0.140000  3.906667   
1      七台河市     2309.0  2017     12 -15.577419 -10.764516  0.116129  3.796774   
2       三亚市     4602.0  2017      1  19.748387  22.945161  0.154839  7.296774   
3       三亚市     4602.0  2017     12  18.596774  21.645161  0.412903  7.741935   
4       三明市     3504.0  2017      1  13.222581  15.000000  1.538710  1.422581   

   sunshine   humidity  
0  4.066667  64.100000  
1  3.580645  62.161290  
2  4.519355  87.580645  
3  4.954839  81.516129  
4  3.225806  78.354839  
(336, 10)
  city_name  city_code  year  month       temp      gtemp      rain      wind  \
0      七台河市     2309.0  2017      1 -13.916667  -9.020000  0.140000  3.906667   
2       三亚市     4602.0  2017      1  19.748387  22.945161  0.154839  7.296774   
4       三明市     3504.0  2017      1  13.222581  15.000000  1.538710  1.422581   
6      三门峽市     411